<a href="https://colab.research.google.com/github/Giffy/Mobility_AIBCN/blob/master/2_Telefonia_Cat_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Install libraries (if required)

In [1]:
# Instalar PyMongo
! python -m pip install pymongo

##  Connect to Google Drive

In [2]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

Connection to Google drive successful


##  Load data

In [3]:
check_file = !if [ -f 'GDrive/My Drive/coverageCatalunya.tar.gz' ]; then echo "1" ; else echo "0"; fi


if (check_file[0] is '0' ):
  print ('Dataset not available in Google Drive: downloading dataset ... \n')
  # If datafile is not in Google Drive the download it
  # Download Open Data from 'Dades obertes catalunya': https://analisi.transparenciacatalunya.cat/browse?q=cobertura%20M%C3%B2bil
  # It dowloads a 1.7Gb csv file  
  !wget "https://analisi.transparenciacatalunya.cat/api/views/g9ma-vbt8/rows.csv?accessType=DOWNLOAD"
  !mv rows* coverageCatalunya.csv
  !tar -czvf coverageCatalunya.tar.gz coverageCatalunya.csv
  # Backup data to Google drive
  !mv coverageCatalunya.tar.gz GDrive/My\ Drive/

  
if (check_file[0] is '1' ):
  print ('Dataset available in Google Drive: inflating dataset ... ')
  !tar -xvf GDrive/My\ Drive/coverageCatalunya.tar.gz coverageCatalunya.csv

Dataset available in Google Drive: inflating dataset ... 
coverageCatalunya.csv


In [0]:
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --db catalunya --collection mobile --type csv --file coverageCatalunya.csv --drop --headerline

##  Connect to MongoDB database

In [0]:
import pymongo
from pymongo import MongoClient
# uri (uniform resource identifier) defines the connection parameters 
uri = 'localhost:27017'
# start client to connect to MongoDB server 
client = MongoClient( uri )

In [0]:
db = client.catalunya                           ##  Seleccionamos la base de datos con nombre "movil"
db.list_collection_names()                      ##  Listamos las colecciones disponibles
collection = db.mobile

##  Database Visualization

In [7]:
print (list (collection.find().limit(1)[1]))
print ('Total documents: ' + str(  collection.count_documents({'_id' : {'$ne': None }}) ))

['_id', 'timestamp_', 'Date', 'Hora_', 'Lat', 'Long_', 'Senyal', 'Xarxa', 'Operador', 'status', 'Desc_', 'net_type', 'speed', 'satellites', 'precision1', 'provider', 'Activitat', 'downloadSpeed', 'uploadSpeed', 'MUNICIPI', 'NOM_MUNI']
Total documents: 11744914


In [0]:
attributes = ['Hora_', 'Activitat', 'Year', 'Month', 'Day', 'Weekday', 'Hour']
attributes = ['Senyal', 'Xarxa', 'Operador', 'status', 'Desc_', 'net_type', 'speed', 'precision1', 'provider', 'Activitat', 'NOM_MUNI','Carrier']

for attribute in attributes:
  if (attribute != '_id'):
    print ( attribute + " "+ str( collection.distinct( attribute )))

## Data modification


In [9]:
list(collection.find().limit(1))

[{'Activitat': 'UNKNOWN',
  'Date': '17/10/2015 01:54:12 AM',
  'Desc_': 'STATE_EMERGENCY_ONLY',
  'Hora_': '01:54:12',
  'Lat': 41.38734,
  'Long_': 2.19812,
  'MUNICIPI': 80193,
  'NOM_MUNI': 'Barcelona',
  'Operador': 'ONO',
  'Senyal': 24,
  'Xarxa': 'movistar',
  '_id': ObjectId('5c23613498a81066a4429c77'),
  'downloadSpeed': '',
  'net_type': '3G',
  'precision1': 43,
  'provider': 'fused',
  'satellites': 0,
  'speed': 1.4,
  'status': 2,
  'timestamp_': 1445039652105,
  'uploadSpeed': ''}]

### Define Area of study

1- Lat: 41.453045    Long: 2.037423

2- Lat: 41.453045    Long: 2.256154
<br><br>

3- Lat: 41.338663    Long: 2.037423

4- Lat: 41.338663    Long: 2.256154

In [10]:
delete = collection.delete_many({'Long_': {"$gt" : 2.256154 }}) 
delete.deleted_count

2782972

In [11]:
delete = collection.delete_many({'Long_': {"$lt" : 2.037423 }}) 
delete.deleted_count

4619737

In [12]:
delete = collection.delete_many({'Lat': {"$gt" : 41.453045 }}) 
delete.deleted_count

1395474

In [13]:
delete = collection.delete_many({'Lat': {"$lt" : 41.338663 }}) 
delete.deleted_count

146115

### Deletion of no relevant fields :  downloadSpeed, uploadSpeed, satellites, MUNICIPI, etc...

In [14]:
filters = {'_id' : {'$ne': None }}
fields = {'$unset': {'downloadSpeed': 1, 'uploadSpeed': 1, 'satellites': 1, 'MUNICIPI': 1, 'speed': 1,
                     'status': 1, 'Senyal':1 , 'Desc_':1, 'provider': 1, 'timestamp_': 1, 'net_type': 1,
                     'NOM_MUNI': 1, 'Operador': 1, 'Xarxa': 1, 'precision1': 1}}
 
collection.update_many( filters ,fields)

In [15]:
list( collection.find().limit(1) )

[{'Activitat': 'UNKNOWN',
  'Date': '17/10/2015 01:54:12 AM',
  'Hora_': '01:54:12',
  'Lat': 41.38734,
  'Long_': 2.19812,
  '_id': ObjectId('5c23613498a81066a4429c77')}]

### Add Year, Month, Day, Weekday, Hour  fields


In [16]:
from bson.objectid import ObjectId
import datetime
import sys

def round_coord(x, base=0.005):
     return round(base*round(x/base), 4)

consulta_tmp = (collection.find({},{'_id','Date','Hora_','Lat','Long_'}))
count = 0
num_docs = collection.count_documents({})
for item in consulta_tmp:
    pk = str(item['_id'])
    datee = str(item['Date'])
    day = int(datee.split('/')[0])
    month = int(datee.split('/')[1])
    year = int(datee.split('/')[2].split(' ')[0])
    horaa = str(item['Hora_'])
    hora = int(horaa.split(':')[0])
    weekday = datetime.datetime(year, month, day).isoweekday()
    lat = float(item['Lat'])
    long = float(item['Long_'])
    lat = round_coord(lat)
    long = round_coord(long)    
    filters = { '_id' : ObjectId(pk) }   
    fields = {'$set': {'Year': year, 'Month' : month ,'Day' : day , 'Weekday' : weekday , 'Hour' : hora, 'Latitude' : lat , 'Longitude' : long }}
    collection.update_one( filters ,fields)
    count += 1 
    if (count%50):    
      b = round((count*100 / num_docs),1)
      sys.stdout.write('\r'+ str(b) + "%")

100.0%

In [17]:
list(collection.find().limit(1))

[{'Activitat': 'UNKNOWN',
  'Date': '17/10/2015 01:54:12 AM',
  'Day': 17,
  'Hora_': '01:54:12',
  'Hour': 1,
  'Lat': 41.38734,
  'Latitude': 41.385,
  'Long_': 2.19812,
  'Longitude': 2.2,
  'Month': 10,
  'Weekday': 6,
  'Year': 2015,
  '_id': ObjectId('5c23613498a81066a4429c77')}]

In [18]:
filters = {'_id' : {'$ne': None }}
fields = {'$unset': {'Date' : 1, 'Lat': 1, 'Long_': 1, 'Hora_':1}}

collection.update_many( filters ,fields)

## Export to CSV

In [19]:
# Export to CSV
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoexport --db catalunya --collection mobile --type csv --out coverageCat.csv --fields Activitat,Day,Hour,Latitude,Longitude,Month,Weekday,Year,_id

2018-12-26T12:10:47.699+0000	connected to: localhost
2018-12-26T12:11:46.890+0000	exported 2800616 records


In [20]:
!tar -czvf coverageCat.tar.gz coverageCat.csv
!mv coverageCat.tar.gz GDrive/My\ Drive/

coverageCat.csv
